In [ ]:
import requests
import pandas as pd
import time
from typing import Dict, List, Optional

In [ ]:
class Scraper:
    def __init__(self):
        self.base_url = "https://fbrapi.com"
        self.api_key = None
        self.premier_league_id = 9

    def generate_api_key(self) -> str:
        try:
            response = requests.post(f"{self.base_url}/generate_api_key")
            response.raise_for_status()
            api_key = response.json()['api_key']
            self.api_key = api_key
            print(f"API Key generated successfully: {api_key}")
            return api_key
        except requests.exceptions.RequestException as e:
            print(f"Error generating API key: {e}")
            return None